https://confluence.ecmwf.int/display/GRIB/index.py

Exaple Grib data


In [3]:
import traceback
import sys,os
 
from gribapi import *
 
INPUT='../../data/index.grib'
VERBOSE=1 # verbose error reporting
 
def product(*args, **kwds):
    # product('ABCD', 'xy') --> Ax Ay Bx By Cx Cy Dx Dy
    # product(range(2), repeat=3) --> 000 001 010 011 100 101 110 111
    pools = map(tuple, args) * kwds.get('repeat', 1)
    result = [[]]
    for pool in pools:
        result = [x+[y] for x in result for y in pool]
    for prod in result:
        yield tuple(prod)
 
def example():
    index_keys = ["shortName","level","number","step"]
    index_file = "my.idx"
 
    iid = None
 
    if (os.path.exists(index_file)):
        iid = grib_index_read(index_file)
    else:
        iid = grib_index_new_from_file(INPUT,index_keys)
 
        # multiple files can be added to an index:
        # grib_index_add_file(iid,"grib file to add")
 
        grib_index_write(iid,index_file)
 
    index_vals = []
 
    for key in index_keys:
        print "%sSize=%d" % (
            key,
            grib_index_get_size(iid,key)
        )
 
        key_vals = grib_index_get(iid,key)
        print " ".join(key_vals)
 
        index_vals.append(key_vals)
 
    for prod in product(*index_vals):
        for i in range(len(index_keys)):
            grib_index_select(iid,index_keys[i],prod[i])
 
        while 1:
            gid = grib_new_from_index(iid)
            if gid is None: break
            print " ".join(["%s=%s" % (key,grib_get(gid,key)) for key in index_keys])
            grib_release(gid)
 
    grib_index_release(iid)
     
 
def main():
    try:
        example()
    except GribInternalError,err:
        if VERBOSE:
            traceback.print_exc(file=sys.stderr)
        else:
            print >>sys.stderr,err.msg
 
        return 1
 
if __name__ == "__main__":
    sys.exit(main())

SyntaxError: invalid syntax (<ipython-input-3-f68f3079e7b7>, line 38)

https://github.com/erdc/grib_api/blob/master/examples/python/samples.py

In [4]:
import traceback
import sys

from gribapi import *

INPUT='../../data/tp_ecmwf.grib'
OUTPUT='out.grib'
VERBOSE=1 # verbose error reporting

def example():
    sample_id = grib_new_from_samples("regular_ll_sfc_grib1")
    fin = open(INPUT)
    fout = open(OUTPUT,'w')

    keys = {
        'dataDate':20080104,
        'startStep':0,
        'endStep':12,
        'stepType':'accum',
        'table2Version':2,
        'indicatorOfParameter':61,
        'decimalPrecision':2,
        }

    prev_vals = None
    while 1:
        gid = grib_new_from_file(fin)
        if gid is None: break

        curr_vals = grib_get_values(gid)

        if prev_vals is None:
            result = prev_vals = curr_vals
        else:
            try:
                # works with NumPy
                result = curr_vals - prev_vals
            except TypeError: # but not with array
                result = curr_vals
                for i in range(len(result)):
                    result[i] -= prev_vals[i]

            prev_vals = curr_vals
            keys['startStep'] += 12
            keys['endStep'] += 12

        clone_id = grib_clone(sample_id)

        for key in keys:
            grib_set(clone_id,key,keys[key])

        grib_set_values(clone_id,result * 1000)

        grib_write(clone_id,fout)

        grib_release(gid)

    fin.close()
    fout.close()

def main():
    try:
        example()
    except GribInternalError,err:
        if VERBOSE:
            traceback.print_exc(file=sys.stderr)
        else:
            print >>sys.stderr,err.msg

        return 1

if __name__ == "__main__":
sys.exit(main())

SyntaxError: invalid syntax (<ipython-input-4-99a12c818c60>, line 64)

https://gis.stackexchange.com/questions/173775/how-to-extract-specific-information-from-grib-files

In [5]:
import traceback
import sys

# hopefully the only external package I need
from gribapi import *

# file with all possible 'messages'
INPUT = '2015121406_gfs.t06z.pgrb2.0p25.f000'
VERBOSE=1 # verbose error reporting

def example():
    f = open(INPUT)

    # location we are interested in
    lat, lon = 64.1353, -21.8952

    while 1:
        # STEP 1:
        # open downloaded GRIB2 file
        gid = grib_new_from_file(f)
        if gid is None: break

        # define the iterator (which is throughout the program the same?)
        iterid = grib_iterator_new(gid,0)

        # get the result for the nearest location
        nearest = grib_find_nearest(gid, lat, lon)[0]

        while 1:
            # STEP 2:
            # ???
            #
            # the loop goes through the whole file
            # instead just selecting messages we need beforehand...
            # fictional function which selects
            # 'TMP' (temperature) on 'surface' messages only
            # without need to iterate all of them:
            # 
            # result = grib_select_specific_message(gid, 'TMP', 'surface')

            result = grib_iterator_next(iterid)
            if not result: break

        # STEP 3:
        # profit!
        # result variable returns numbers which I process in any way I need, yay!

        # more undocumented stuff,
        # put here just because examples do it the same way
        grib_iterator_delete(iterid)
        grib_release(gid)

    f.close()

# main program function
def main():
    try:
        example()
    except GribInternalError,err:
        if VERBOSE:
            traceback.print_exc(file=sys.stderr)
        else:
            print >>sys.stderr,err.msg

        return 1

# run the program
if __name__ == "__main__":
    sys.exit(main())

SyntaxError: invalid syntax (<ipython-input-5-7904e0d5c088>, line 59)

In [8]:
 import pygrib

#     grbs= pygrib.open("my_file.grb")
#     grbs= pygrib.open("C:\Users\Mhdella\Downloads\USTemperatureForecast.grb2")
    grbs= pygrib.open("../../USTemperatureForecast.grb2")

    # use grbs.select to select the grids you are interested in (shortName, typeOfLevel, level=500, marsParam, dataDate, dataTime, ...)

    DATA=np.array(grbs.select(marsParam=my_param,dataDate=my_date,dataTime=my_time))

    # DATA will contain 3 arrays
    # DATA[0] for values
    # DATA[1] for longitudes
    # DATA[2] for latitudes

    # from the "values" array, extract in lon and lat
    DATA1=DATA[0].data(lat1=my_y1,lat2=my_y2,lon1=my_x1,lon2=my_x2))

IndentationError: unexpected indent (<ipython-input-8-b009f56f325a>, line 5)

In [12]:
import pygrib
# grbs = pygrib.open('sampledata/gfs.grb')

ModuleNotFoundError: No module named 'pygrib'